In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import os
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_for_WarmEarly_Kayla_20190220'

In [2]:
inclusion_stores=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_for_WarmEarly_Kayla_20190220/Master Store by Attribute Listing Feb 2019.xlsm",dtype=str)
del inclusion_stores['Unnamed: 0']
inclusion_stores=inclusion_stores[inclusion_stores['EWC'].isin(["Warm","Early"])]
inclusion_stores=inclusion_stores['Store'].unique().tolist()

In [3]:
store_labels=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_labels=store_labels[['location_id','revenue_flag','zip']]
store_labels=store_labels[store_labels['revenue_flag'].isin(["P","S"])]
store_labels=store_labels[store_labels['location_id'].isin(inclusion_stores)]
len(store_labels['location_id'].unique())

774

In [4]:
new_stores=[x for x in inclusion_stores if x not in store_labels['location_id'].unique().tolist()]
len(new_stores)

32

In [5]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190201-133832-957.txt",dtype=str,sep="|")
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

# store_list['zip']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list=store_list[['location_id','latitude_meas','longitude_meas']]

In [6]:
output_part_1=pd.merge(store_labels,store_list,on="location_id",how="left")
output_part_1=output_part_1.reset_index()
del output_part_1['index']
output_part_1['dist_miles']=np.nan

for i in range(len(output_part_1)):
    zip_cd=output_part_1['zip'][i]
    store_centers=[output_part_1['latitude_meas'][i],output_part_1['longitude_meas'][i]]

    if zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_centers,miles=True)
    else:
        dist=np.nan
    output_part_1['dist_miles'][i]=dist

output_part_1.head(5)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,location_id,revenue_flag,zip,latitude_meas,longitude_meas,dist_miles
0,1001,P,37110,35.699926,-85.781388,0.774369
1,1001,S,37357,35.699926,-85.781388,10.158042
2,1001,S,37166,35.699926,-85.781388,16.049509
3,1001,S,38581,35.699926,-85.781388,9.007219
4,1001,S,37190,35.699926,-85.781388,17.052225


In [7]:
new_stores_1=store_list[store_list['location_id'].isin(new_stores)]
new_stores_1.shape
new_stores_2=[x for x in new_stores if x not in new_stores_1['location_id'].unique().tolist()]
new_stores_2

['4674', '5370', '4693']

In [8]:
new_stores_2_dict={}
new_stores_2_dict.update({"4674":[38.7211536,-121.2927469]})
new_stores_2_dict.update({"4693":[45.6780365,-122.5548524]})
new_stores_2_dict.update({"5370":[27.3400793,-82.4995067]})
new_stores_2=pd.DataFrame({"location_id":list(new_stores_2_dict.keys())},index=[0,1,2])
new_stores_2['latitude_meas']=new_stores_2['location_id'].apply(lambda x: new_stores_2_dict[x][0])
new_stores_2['longitude_meas']=new_stores_2['location_id'].apply(lambda x: new_stores_2_dict[x][1])

In [9]:
new_stores=new_stores_1.append(new_stores_2)
new_stores=new_stores.sort_values(['location_id']).reset_index()
del new_stores['index']

In [10]:
output_part_2=pd.DataFrame()
for i in range(len(new_stores)):
    store_id=new_stores['location_id'][i]
    store_center=[new_stores['latitude_meas'][i],new_stores['longitude_meas'][i]]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":store_id,"revenue_flag":"zips_in_10","zip":zip_cd,"latitude_meas":store_center[0],"longitude_meas":store_center[1],"dist_miles":dist},index=[0])
            output_part_2=output_part_2.append(df)
output_part_2=output_part_2.reset_index()
del output_part_2['index']
output_part_2=output_part_2[output_part_1.columns.tolist()]

In [11]:
output_part_1=output_part_1[(output_part_1['revenue_flag']=="P") | (output_part_1['dist_miles']<=20)]

In [12]:
output_by_store=output_part_1.append(output_part_2)
output_by_store.head(3)

,location_id,revenue_flag,zip,latitude_meas,longitude_meas,dist_miles
0,1001,P,37110,35.699926,-85.781388,0.774369
1,1001,S,37357,35.699926,-85.781388,10.158042
2,1001,S,37166,35.699926,-85.781388,16.049509


In [14]:
output_by_zip=output_by_store[['zip','revenue_flag','dist_miles']]
output_by_zip=output_by_zip.sort_values(["zip",'revenue_flag','dist_miles']).drop_duplicates('zip')

In [15]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_for_WarmEarly_Kayla_20190220/BL_zips_for_806_Warm_Early_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_by_zip=output_by_zip[['zip','revenue_flag']]
output_by_zip.to_excel(writer,"Unique_zips",index=False)
output_by_store=output_by_store.sort_values(["location_id","revenue_flag",'zip'])
output_by_store.to_excel(writer,"zips_by_store",index=False)
writer.save()